
## Analisis Red Social

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import sys
import pandas as pd
#import tensorflow
#import keras
import wordcloud
import nltk
import inspect
import re
import unicodedata

from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import RSLPStemmer #Stemmer for portugese words.

from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.gridspec import GridSpec
#import missingno as msno
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import string
import sys
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,roc_auc_score,roc_curve
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import time
import sys
import math
import itertools
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
#import folium
#from folium.plugins import Fullscreen
#import geopandas as gpd
import json
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')
stop = stopwords.words('portuguese')
#stop.drop('nao') #Stopword already have "Não", just adding this because it's appear on dataframe

## ACA LEO LA BASE DE REDES

In [3]:
#os.chdir("/Users/danielacadavid/Documents/Universidad/Maestria/201920/00_Bases/brazilian-ecommerce/")
os.chdir("C:\\Users\\d.cadavid141\\Documents\\Coisas Daniela\\201920\\201920\\00_Bases\\brazilian-ecommerce\\")
#os.getcwd()
manual=pd.read_excel('Base_manual.xlsx')
original=pd.read_excel('Base_manual.xlsx')

In [4]:
manual.shape

(842, 3)

## Lista de Lemma

In [5]:
lemma_portugues = pd.read_csv('lemmatization_pt.txt',sep="\t", header=None)
lemma_portugues.columns = ["lemma", "palavra"]
lemma_portugues.head()

,lemma,palavra
0,00,iodargírios
1,a,aa
2,aacheniano,aacheniana
3,aacheniano,aachenianas
4,aacheniano,aachenianos


In [6]:
nltk.download('punkt')
stemmer = RSLPStemmer()
import re
import unicodedata

def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def remove_urls (text):
    text = re.sub(r"http\S+", "", text)
    return (text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\d.cadavid141\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
lp=list(lemma_portugues.palavra)
llem=list(lemma_portugues.lemma)
#llem=dict(lemma_portugues.lemma)
#lemma=dict(lemma_portugues)
#llem

In [8]:
from nltk.tokenize import word_tokenize as word_tokenize
stop.remove('não')
#stop.remove('nao')

In [9]:
# Applying RegEx for removing urls and sites
manual.Comment = manual.Comment.apply(remove_urls)

#removing numbers
manual.Comment = manual.Comment.str.replace('\d+', ' ')

#lower cases.
manual.Comment = manual.Comment.apply(lambda x: " ".join(x.lower() for x in x.split()))

#Removing punctuation
manual.Comment = manual.Comment.str.replace('[^\w\s]',' ')

#Removing stopword
manual.Comment = manual.Comment.apply(lambda x: " ".join(x for x in x.split() if x not in stop))

#Tokenmize
manual.Comment = manual.apply(lambda row: word_tokenize(row['Comment']), axis=1)

In [10]:
links = list(manual['Comment'].values)


In [11]:
print(f'Comentario 1:\n')
print(f'{links[0]}\n')
print(f'Comentario 50:\n')
print(f'{links[50]}\n')
#print(f'Comentario 100: \n')
#print(f'{links[100]}\n')

Comentario 1:

['boa', 'tarde', 'fiz', 'compra', 'produto', 'namorada', 'faz', 'meses', 'produto', 'começou', 'desligar', 'sozinho', 'nao', 'quer', 'ligar', 'quero', 'fazer', 'troca', 'reembolso', 'urgente', 'nao', 'chegou', 'prazo']

Comentario 50:

['excelente', 'produto', 'atendeu', 'expectativas', 'recomendo']



In [12]:
lt=list(manual.Comment)

In [13]:
from tqdm import tqdm
getattr(tqdm, '_instances', {}).clear()  # ⬅ add this line

In [14]:
for myList in tqdm(lt):
    for n, i in enumerate(myList):
        if i in lp:
            pos=lp.index(i)
            item2 = llem[pos]
            myList[n]=item2
        else:
            item2=i
            myList[n]=item2

100%|██████████| 842/842 [02:19<00:00,  6.06it/s]


In [15]:
manual.Comment=lt

In [16]:
def strip_accents(list):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
       # for n, i in enumerate(list):
       #     text = unicode(n, 'utf-8')
       #     text = unicodedata.normalize('NFD', text)
       #     text = text.encode('ascii', 'ignore')
       #     text = text.decode("utf-8")
       #     list[n]=text
        text = ' '.join([str(item) for item in list ])
        text = unicode(text, 'utf-8')
        
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    
    return str(text)

In [17]:
#manual.Comment=lt

#removing accentuation
manual.Comment = manual.Comment.apply(strip_accents)

#Tokenize again
manual.Comment = manual.apply(lambda row: word_tokenize(row['Comment']), axis=1)


In [18]:
links = list(manual['Comment'].values)
print(f'Comentario 1:\n')
print(f'{links[0]}\n')
print(f'Comentario 50:\n')
print(f'{links[50]}\n')
print(f'Comentario 100: \n')
#print(f'{links[100]}\n')

Comentario 1:

['bom', 'tardar', 'fazer', 'comprar', 'produto', 'namorado', 'fazer', 'mes', 'produto', 'comecar', 'desligar', 'so', 'nao', 'querer', 'ligar', 'querer', 'fazer', 'trocar', 'reembolsar', 'urgente', 'nao', 'chegar', 'prazo']

Comentario 50:

['excelente', 'produto', 'atender', 'expectativa', 'recomendar']

Comentario 100: 



In [19]:
org = list(original['Comment'].values)
print(f'Comentario 1:\n')
print(f'{org[0]}\n')
print(f'Comentario 50:\n')
print(f'{org[50]}\n')
print(f'Comentario 100: \n')
#print(f'{org[100]}\n')

Comentario 1:

Boa tarde! Fiz a compra do produto para a minha namorada faz 4 meses e o produto começou a desligar sozinho e nao quer ligar. Quero fazer a troca ou reembolso urgente.  Nao chegou no prazo

Comentario 50:

Excelente produto atendeu minhas expectativas, eu recomendo!

Comentario 100: 



In [20]:
manual.to_csv("C:\\Users\\d.cadavid141\\Documents\\Coisas Daniela\\201920\\201920\\00_Bases\\brazilian-ecommerce\\data_tratada_manual_grande.csv")
